<a href="https://www.kaggle.com/code/dascient/daily-crypto-buy-sell-decision-maker?scriptVersionId=141042657" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Updates daily - 08:00 CST. 
## An automated solution for lazy Binance.US traders that depend solely on technical analysis. 

### You're welcome :)

**Scheduled notebook**

This notebook is scheduled to run everyday at 08:00 CST to collect 300-day historical price actions on all Binance.US tradeable assets. It employs a plethoric set of technical indicators such as a kalman filter modeler, ichimoku forecasting condtionals, as well as, other complex algorithms in order to analyze and identify profitable entry & exit points.

### *POTENTIAL UPCOMING OUTAGE*
Due to Binance US having made some important API changes. Certain consequences may impact this notebook's ability to query historical transaction data for USD trading pairs, including trades and orders, that were executed before our system upgrade on February 5, 2023 at 9 p.m. PST / midnight EST.

To query transaction information for historical trades and orders, involving USD trading pairs, executed before the planned system upgrade, we will need to begin using the USD4 symbol. For example: "BTCUSD4".
To query transaction information for trades and orders, involving USD trading pairs, executed after the planned system upgrade, we would need to continue using the USD symbol. For example: “BTCUSD”.

***PLEASE STANDBY FOR MINOR UPDATES***

In [33]:
# login to binance
from IPython.display import clear_output

!pip install ccxt
!pip install schedule
!pip install pykalman
!pip install pandas_ta
!pip install plotly

import ccxt,schedule,warnings,time,ast
from pykalman import KalmanFilter
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from dateutil.tz import tzlocal
from rich import print, pretty
from datetime import datetime
from random import randint
from random import seed
import pandas_ta as ta
import pandas as pd
import numpy as np
pretty.install()
import os

clear_output()

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [34]:
# this key pair only works for me, but if you'd like to run on your own binance api you can obtain yours by following:
# https://youtu.be/Q4Lt1KulkOQ

ccxt.binanceus({ 'options':{ 'adjustForTimeDifference':True}})
exchange = ccxt.binanceus({
"apiKey": "n0Oiypu18lqOHAkROTpYRUBNaoFWzIomdNk36S8g5xMPOUwhp8ZozRg4eMQ32Cl2",
"secret": "BzXJTwRBZuRNUViZZWoAvTdykXBl2wvsQ6hyUWI4deEJiQdR48u0WNqOBFY4GdT8",
'enableRateLimit': True})

# Calculate Indicators

In [38]:
# indicators + trade signals
def calculate_indicator(symbol):

    bars = exchange.fetch_ohlcv(symbol, timeframe='1d', limit=300)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize(None)
    
    close = df['close'][len(df)-1]
    low = df['low'][len(df)-1]
        
        
    # Construct a Kalman filter
    kf = KalmanFilter(transition_matrices = [1],    # The value for At. It is a random walk so is set to 1.0
                      observation_matrices = [1],   # The value for Ht.
                      initial_state_mean = 0,       # Any initial value. It will converge to the true state value.
                      initial_state_covariance = 1, # Sigma value for the Qt in Equation (1) the Gaussian distribution
                      observation_covariance=1,     # Sigma value for the Rt in Equation (2) the Gaussian distribution
                      transition_covariance=.01)    # A small turbulence in the random walk parameter 1.0
    # Get the Kalman smoothing
    state_means, _ = kf.filter(df['close'].values)
    # Call it kf_mean
    df['kf_mean'] = np.array(state_means)
    kalman = df.kf_mean[len(df)-1]
    aboveKalman = low > kalman
    
    
    # exponential moving averages 
    ema_14 = df.ta.ema(14, append=True)[-1:].reset_index(drop=True)[0]
    ema_91 = df.ta.ema(91, append=True)[-1:].reset_index(drop=True)[0]
    ema_125 = df.ta.ema(125, append=True)[-1:].reset_index(drop=True)[0]
    ema_crossover = ema_14 > ema_91
    
    
    # lower/upper 14-day bollinger bands for mean reversion
    bbl_14 = df.ta.bbands(length=14, append=True)[['BBL_14_2.0']].tail(1).values[0][0]
    bbu_14 = df.ta.bbands(length=14, append=True)[['BBU_14_2.0']].tail(1).values[0][0]
    bband_buy = close < bbl_14
    bband_sell = close > bbu_14
    
    
    # ichimoku 9 & 26-day forecasts 
    # https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#ta.trend.IchimokuIndicator
    isa_9 = df.ta.ichimoku()[1]['ISA_9'].tail(1).values[0] # help(ta.ichimoku)
    isb_26 = df.ta.ichimoku()[1]['ISB_26'].tail(1).values[0]

    
    # archer ma 
    # https://github.com/twopirllc/pandas-ta#general
    amat = (df.ta.amat()['AMATe_LR_8_21_2'].tail(1).values[0] == 1)

    
    # rsi
    rsi = df.ta.rsi()[len(df)-1]
    rsi_buy = rsi < 30
    rsi_sell = rsi > 70

    
    # choppy
    # https://github.com/twopirllc/pandas-ta#trend-18
    try: 
        chop = "{:.2f}".format(df.ta.chop()[len(df.ta.chop())-1]) 
    except RunTimeWarning:
        chop = 0


    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    # signal
    #buy = (close < isa_9) & (close < isb_26) & amat & rsi_buy & bband_buy & aboveKalman
    buy = amat & ema_crossover & aboveKalman
    
    #sell = (close > isa_9) & (close > isb_26) & ~amat & rsi_sell & bband_sell & ~aboveKalman
    sell = ~amat & ~ema_crossover & ~aboveKalman

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

    return df, symbol, close, isa_9, isb_26, chop, rsi, amat, ema_crossover, buy, sell, aboveKalman


# plotter
def plot(symbol):

    df = calculate_indicator(symbol)[0]

    fig = go.Figure(go.Candlestick(x=df.index,
                                   open=df['open'],
                                   high=df['high'],
                                   low=df['low'],
                                   close=df['close'],
                                   name=f'{symbol}'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['kf_mean'], 
                             opacity=0.7, 
                             line=dict(color='orange', width=2), 
                             name='Kalman Filter'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['EMA_91'], 
                             opacity=0.7, 
                             line=dict(color='purple', width=2), 
                             name='EMA-91'))
    
    #fig.update_layout(title=f'{symbol}')
    fig.update_layout(xaxis_rangeslider_visible=False)
    return fig.show()


# color in dataframe
def color_boolean(val):
    color =''
    if val == True:
        color = 'lightgreen'
    elif val == False:
        color = 'pink'
    elif type(val) == str or float or int:
        color = 'lightblue'
    return 'background-color: %s' % color


# Prepare Data

In [39]:
today = pd.Timestamp(datetime.now()).strftime("%Y-%m-%d")
print(f"Calculating indicators for today's date: {today}.")

Calculating indicators for today's date: 2023-08-26.

In [40]:
# load symbols + initiate dataframe
results = []
symbols = exchange.fetchTickers().keys()

# iteration
for symbol in symbols:
    try:
        output = calculate_indicator(symbol)
        results.append({'Symbol':output[1],
                        'Buy Signal':output[9],
                        'Sell Signal':output[10],
                        'Date':today,
                        'Close':output[2],
                        'Ichimoku 9-Day Forecast':output[3],
                        'Ichimoku 26-Day Forecast':output[4],
                        'Choppiness (%)':output[5],
                        'RSI':output[6],
                        'Archer MA Trending':output[7],
                        'EMA14 > EMA91':output[8],
                        'Low > Kalman':output[11],
                        })
    except:
        pass
    
# frame & clean
results = pd.DataFrame(results)
results['Tick'] = [v.split('/')[0] for i,v in results.Symbol.items()]
results['Deno'] = [v.split('/')[1] for i,v in results.Symbol.items()]

# set pandas display to respect number of symbols in results
size = len(results)
pd.set_option('display.max_rows', size)

In [41]:
# get only 10 top buy & 10 top sell
df_top = results.sort_values(['Buy Signal','EMA14 > EMA91','Archer MA Trending'],ascending=False)
top10 = df_top[df_top["Tick"].str.contains("USD")==False].sort_values(['RSI'],ascending=True).drop_duplicates('Tick').head(10).reset_index(drop=True)
top20 = df_top[df_top["Tick"].str.contains("USD")==False].sort_values(['RSI'],ascending=True).drop_duplicates('Tick').head(20).reset_index(drop=True)

df_bottom = results.sort_values(['Sell Signal','Archer MA Trending','RSI'],ascending=False)
bottom10 = df_bottom[df_bottom["Tick"].str.contains("USD")==False].drop_duplicates('Tick').head(10).reset_index(drop=True)
bottom20 = df_bottom[df_bottom["Tick"].str.contains("USD")==False].drop_duplicates('Tick').head(20).reset_index(drop=True)

# Buying Opportunities

In [42]:
if any(top20['Buy Signal'] == True): 
    print("The charts below are assets that were identified as reasonable 'entry points' - sorted by ema crossovers, trends, & rsi values.")
    print("Great entry points for:")
    [print(y) for y in top10[top10['Buy Signal'] == True].Symbol.items()]
    
if all(top20['Buy Signal'] == False): 
    print("No recommended long positions today. Showing results sorted by significance of crossovers, trends, & rsi values.")
    

No recommended long positions today. Showing results sorted by significance of crossovers, 
trends, & rsi values.

In [43]:
top20.style.applymap(color_boolean)

,Symbol,Buy Signal,Sell Signal,Date,Close,Ichimoku 9-Day Forecast,Ichimoku 26-Day Forecast,Choppiness (%),RSI,Archer MA Trending,EMA14 > EMA91,Low > Kalman,Tick,Deno
0,BOSON/USDT,False,True,2023-08-26,0.101800,0.118550,0.135350,4.75,21.334244,False,False,False,BOSON,USDT
1,CRV/USDT,False,True,2023-08-26,0.460200,0.520000,0.649500,38.91,21.465122,False,False,False,CRV,USDT
2,GAL/USDT,False,True,2023-08-26,1.001000,1.056250,1.100000,22.34,22.142103,False,False,False,GAL,USDT
3,BOND/USDT,False,True,2023-08-26,2.162000,2.555500,2.873500,30.62,22.546139,False,False,False,BOND,USDT
4,LRC/BTC,False,True,2023-08-26,0.000009,0.000010,0.000012,38.83,22.773935,False,False,False,LRC,BTC
5,SAND/USDT,False,True,2023-08-26,0.321700,0.367100,0.403750,29.10,23.022348,False,False,False,SAND,USDT
6,REQ/USDT,False,True,2023-08-26,0.060200,0.068900,0.081500,16.06,23.257758,False,False,False,REQ,USDT
7,FIL/USDT,False,True,2023-08-26,3.227000,3.530750,3.914000,31.65,23.677360,False,False,False,FIL,USDT
8,POLYX/USDT,False,True,2023-08-26,0.100000,0.117025,0.115850,25.81,23.775084,False,False,False,POLYX,USDT
9,AVAX/USDT,False,True,2023-08-26,10.050000,11.057500,12.840000,33.17,23.914665,False,False,False,AVAX,USDT


In [44]:
# plots for top10
[plot(symbol) for i,symbol in top10.Symbol.items()]

[None, None, None, None, None, None, None, None, None, None]

# Selling Opportunities

In [45]:
if any(bottom20['Sell Signal'] == True): 
    print("The charts below are assets that were identified as reasonable 'exit points' - sorted by crossovers, trends, & rsi values.")
    print("Best selling points are at:")
    [print(y) for y in bottom10[bottom10['Sell Signal'] == True].Symbol.items()]
        
    
if all(bottom20['Sell Signal'] == False): 
    print("No recommended short positions today. Showing results sorted by significance of crossovers, trends, & rsi values.")
    

The charts below are assets that were identified as reasonable 'exit points' - sorted by 
crossovers, trends, & rsi values.

Best selling points are at:

(0, 'HNT/USD4')

(1, 'SANTOS/USD4')

(2, 'SOL/BUSD')

...

(5, 'QNT/USD')

(6, 'LAZIO/USD')

(7, 'DIA/USD')

(8, 'NEAR/BUSD')

(9, 'ROSE/USD')

In [46]:
bottom20.style.applymap(color_boolean)

,Symbol,Buy Signal,Sell Signal,Date,Close,Ichimoku 9-Day Forecast,Ichimoku 26-Day Forecast,Choppiness (%),RSI,Archer MA Trending,EMA14 > EMA91,Low > Kalman,Tick,Deno
0,HNT/USD4,False,True,2023-08-26,2.796400,2.749900,2.472050,70.78,52.918649,False,False,False,HNT,USD4
1,SANTOS/USD4,False,True,2023-08-26,5.685000,5.744000,5.151000,57.02,52.384632,False,False,False,SANTOS,USD4
2,SOL/BUSD,False,True,2023-08-26,21.070000,20.677500,21.570000,56.00,50.354945,False,False,False,SOL,BUSD
3,MANA/BUSD,False,True,2023-08-26,0.596900,0.576325,0.653600,48.16,49.633942,False,False,False,MANA,BUSD
4,ALPINE/USDT,False,True,2023-08-26,2.026000,2.170550,2.087550,54.06,49.377155,False,False,False,ALPINE,USDT
5,QNT/USD,False,True,2023-08-26,105.000000,60.325000,60.600000,29.84,49.187335,False,False,False,QNT,USD
6,LAZIO/USD,False,True,2023-08-26,1.890100,2.367775,2.869200,65.33,48.230874,False,False,False,LAZIO,USD
7,DIA/USD,False,True,2023-08-26,0.250000,0.413000,0.505000,53.02,47.748391,False,False,False,DIA,USD
8,NEAR/BUSD,False,True,2023-08-26,1.971000,2.791500,3.580500,50.93,47.587177,False,False,False,NEAR,BUSD
9,ROSE/USD,False,True,2023-08-26,0.047800,0.037780,0.035725,41.53,47.171608,False,False,False,ROSE,USD


In [47]:
# plots for bottom10
[plot(symbol) for i,symbol in bottom10.Symbol.items()]

[None, None, None, None, None, None, None, None, None, None]

# Save in CSV Format

In [48]:
# save to csv
datestr = datetime.now().strftime("%Y%m%d")
results.to_csv('DailySignals_{}.csv'.format(datestr),index=False)
print("CSV now in Outputs </kaggle/working/...>")

CSV now in Outputs </kaggle/working/...>

# Supplementary

In [ ]:
# Prints the indicators and utility functions
results.ta.indicators()

# Returns a list of indicators and utility functions
ind_list = results.ta.indicators(as_list=True)

# Returns a list of the indicators and utility functions that are not in the excluded list
smaller_list = results.ta.indicators(exclude=["cg", "pgo", "ui"], as_list=True)

# Trade Automation Checklist

1. Retrieve currency balances: USD, USDT, BUSD, BTC, ETH, ...
2. Use top10 & bottom10 as list of tradeable symbols.
3. Trade cash balance divided by number of symbols in long positions. trade_amount = (balance / len(top10.loc(top10.Long == True))
4. React to bottom10 short positions first. Sell trade_amount of each symbol in bottom10.loc(bottom10.Short == True)
5. For top10 long positions. Trade with Deno > trade_amount. 

In [ ]:
# en fin